## Introduction

#### Analysis feature on this competitions 
* Analysis all feature that used and do data visualize
* Hep to find the key features on this competitions

## Training  kernel ==> https://www.kaggle.com/super13579/revisit-one-hot-ranking

### Version 1 ( LB = 1.5057)
* Process Date feature (add day, week, hour, "revisit time")
* Totals feature don't do anything, only fillna(0)
* Do one hot encoding for categorical feature that have unique value <15
* Do ranking encoding for categorical feature that have unique value >15

**Data Fields**
* fullVisitorId- A unique identifier for each user of the Google Merchandise Store.
* channelGrouping - The channel via which the user came to the Store.
* date - The date on which the user visited the Store.
* device - The specifications for the device used to access the Store.
* geoNetwork - This section contains information about the geography of the user.
* sessionId - A unique identifier for this visit to the store.
* socialEngagementType - Engagement type, either "Socially Engaged" or "Not Socially Engaged".
* totals - This section contains aggregate values across the session.
* trafficSource - This section contains information about the Traffic Source from which the session originated.
* visitId - An identifier for this session. This is part of the value usually stored as the _utmb cookie. This is only unique to the user. For a completely unique ID, we should use a combination of fullVisitorId and visitId.
* visitNumber - The session number for this user. If this is the first session, then this is set to 1.
* visitStartTime - The timestamp (expressed as POSIX time).

## Load data
* import module that data analysis need to used
* load original data to see feature and data type

★There are some feature have json object

In [ ]:
import os
print(os.listdir("../input"))
import numpy as np 
import pandas as pd 
import json
import bq_helper
from pandas.io.json import json_normalize
import warnings
import gc
gc.enable()
warnings.filterwarnings("ignore")


## Flatten feature with json object
* There are json objects in some features.
* We used this function written by julian in his kernel to flat these features.

https://www.kaggle.com/julian3833/1-quick-start-read-csv-and-flatten-json-fields/notebook.

In [ ]:
json_columns = ['device', 'geoNetwork','totals', 'trafficSource']
def load_dataframe(filename):
    path = "../input/" + filename
    df = pd.read_csv(path, converters={column: json.loads for column in json_columns}, 
                     dtype={'fullVisitorId': 'str'})
    
    for column in json_columns:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}_{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    return df

### Show the flatten data

In [ ]:
train = load_dataframe("ga-customer-revenue-prediction/train.csv")
print(train.shape)
train.head()

In [ ]:
test = load_dataframe("ga-customer-revenue-prediction/test.csv")
print(test.shape)
test.head()

## Align test and train features
* We found train set have 2 more feature than test set
* 1 is target featrue, find another feature that not include in test set and remove it on train set

In [ ]:
not_in_test_feature = [i for i in train.columns if i not in test.columns]
print (not_in_test_feature)

In [ ]:
train = train.drop(['trafficSource_campaignCode'], axis = 1)
train.shape,test.shape

## Analysis unique count of each feature and remove only one unique feature
* There are some features that contains only one value "not available in demo dataset" as constant values
* constant value dont have any effective on model prdictions, let's remove it.

* Train Set

In [ ]:
#import module of matplot
import matplotlib.pyplot as plt
import seaborn as sns 
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
from plotly import tools
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy.stats as st
%matplotlib inline

In [ ]:
#analysis count of each features
unique_count_train = []
for i in train.columns:
    unique_count_train.append(len(train[str(i)].value_counts()))
unique_datafram_train = {
    "Feature":train.columns,
    "unique_count":unique_count_train
}
unique_train = pd.DataFrame(unique_datafram_train)
unique_train
#=========================or you can use unique to do this =================================#
#train.nunique()

### Remove constant features

In [ ]:
const_columns_train = unique_train.loc[unique_train['unique_count'] == 1,"Feature"]
const_columns_train = list(const_columns_train)
#=========================or you can use unique to do this =================================#
#const_columns_train = [col for col in train.columns if train[col].nunique() == 1]
print ("Shape before remove constant feature on train set = " + str(train.shape))
train = train.drop(const_columns_train,axis = 1)
print ("Shape after remove constant feature on train set = " + str(train.shape))

* Test Set

In [ ]:
#analysis count of each features
unique_count_test = []
for i in test.columns:
    unique_count_test.append(len(test[str(i)].value_counts()))
unique_datafram_test = {
    "Feature":test.columns,
    "unique_count":unique_count_test
}
unique_test = pd.DataFrame(unique_datafram_test)
unique_test
#=========================or you can use unique to do this =================================#
#test.nunique()

In [ ]:
const_columns_test = unique_test.loc[unique_test['unique_count'] == 1,"Feature"]
const_columns_test = list(const_columns_test)
#=========================or you can use unique to do this =================================#
#const_columns_test = [col for col in test.columns if test[col].nunique() == 1]
print ("Shape before remove constant feature on train set = " + str(test.shape))
test = test.drop(const_columns_test,axis = 1)
print ("Shape after remove constant feature on train set = " + str(test.shape))


## Analysis Date feature
* Extracting new feature from date
* Analysis new feature 

In [ ]:
def extract_new_feature(df): 
    print("Start extract date...")
    df['date'] = pd.to_datetime(df['visitStartTime'], unit='s')
    df['day_of_week'] = df['date'].dt.dayofweek
    df['hour'] = df['date'].dt.hour
    df['day'] = df['date'].dt.day
    df['month'] = df['date'].dt.month
    print("Finished extract date...")
extract_new_feature(train)
extract_new_feature(test)

### Plot Visit of extracted date feature

In [ ]:
t1 = train['day_of_week'].value_counts()
t2 = train['hour'].value_counts()
t3 = train['day'].value_counts()
t4 = train['month'].value_counts()

plt.subplots(figsize=(15,10))
plt.subplot(2,2,1)
plt.title('Visits by day of week',color='b',fontsize=12)
plt.bar(t1.index,t1.values,color='orange')
plt.xticks(t1.index,rotation='horizontal',fontsize=12)
plt.yticks(fontsize=12)

plt.subplot(2,2,2)
plt.title('Visits by hour',color='b',fontsize=12)
plt.bar(t2.index,t2.values,color='pink')
plt.xticks(t2.index,rotation='horizontal',fontsize=10)
plt.yticks(fontsize=12)

plt.subplot(2,2,3)
plt.title('Visits by day',color='b',fontsize=12)
plt.bar(t3.index,t3.values,color='blue')
plt.xticks(t3.index,rotation='horizontal',fontsize=10)
plt.yticks(fontsize=12)

plt.subplot(2,2,4)
plt.title('Visits by month',color='b',fontsize=12)
plt.bar(t4.index,t4.values,color='purple')
plt.xticks(t4.index,rotation='horizontal',fontsize=12)
plt.yticks(fontsize=12)

### Plot total trascations revenue of extracted date feature

In [ ]:
train['totals_transactionRevenue'] = train['totals_transactionRevenue'].astype('float64')
a = train[['day_of_week','totals_transactionRevenue']].groupby('day_of_week').mean().reset_index()
a1 = train[['hour','totals_transactionRevenue']].groupby('hour').mean().reset_index()
a2 = train[['day','totals_transactionRevenue']].groupby('day').mean().reset_index()
a3 = train[['month','totals_transactionRevenue']].groupby('month').mean().reset_index()

plt.subplots(figsize=(15,10))
plt.subplot(2,2,1)
plt.title('Total transactionRevenue mean by day of week',color='b',fontsize=12)
plt.bar(a['day_of_week'],a['totals_transactionRevenue'],color='orange')
plt.xticks(a['day_of_week'],rotation='horizontal',fontsize=12)
plt.yticks(fontsize=12)

plt.subplot(2,2,2)
plt.title('Total transactionRevenue mean by hour',color='b',fontsize=12)
plt.bar(a1['hour'],a1['totals_transactionRevenue'],color='pink')
plt.xticks(a1['hour'],rotation='horizontal',fontsize=10)
plt.yticks(fontsize=12)

plt.subplot(2,2,3)
plt.title('Total transactionRevenue mean by day',color='b',fontsize=12)
plt.bar(a2['day'],a2['totals_transactionRevenue'],color='blue')
plt.xticks(a2['day'],rotation='horizontal',fontsize=10)
plt.yticks(fontsize=12)

plt.subplot(2,2,4)
plt.title('Total transactionRevenue mean by month',color='b',fontsize=12)
plt.bar(a3['month'],a3['totals_transactionRevenue'],color='purple')
plt.xticks(a3['month'],rotation='horizontal',fontsize=12)
plt.yticks(fontsize=12)

### Find the time period of same ID revisit 
idea comes from Aahish kernel :  https://www.kaggle.com/ashishpatel26/future-is-here

In [ ]:
def add_time_period_of_same_ID(df): 
    print("Start add time period feature...")
    df.sort_values(['fullVisitorId', 'date'], ascending=True, inplace=True)
    df['next_revisit_time'] = (
        df['date'] - df[['fullVisitorId', 'date']].groupby('fullVisitorId')['date'].shift(1)
    ).astype(np.int64) // 1e9 // 60 // 60
    df['prev_revisit_time'] = (
        df['date'] - df[['fullVisitorId', 'date']].groupby('fullVisitorId')['date'].shift(-1)
    ).astype(np.int64) // 1e9 // 60 // 60
    print("Finished time periodfeature...")
add_time_period_of_same_ID(train)
add_time_period_of_same_ID(test)

* Plot the epriod time that revisit cause totals_transactionRevenue
* We found that the totals_transactionRevenue are locate at small revisit time portion
### We keep the feature of next and previous revisit time

In [ ]:
b = train[['next_revisit_time','totals_transactionRevenue']].groupby('next_revisit_time').mean().reset_index()
b[b['next_revisit_time']== -2562048] = 0 #Process no revisit ID to 0
b.sort_values(by = "next_revisit_time",ascending=False).fillna(0)
b2 = train[['prev_revisit_time','totals_transactionRevenue']].groupby('prev_revisit_time').mean().reset_index()
b2[b2['prev_revisit_time']== -2562048] = 0
b2.sort_values(by = "prev_revisit_time",ascending=False).fillna(0)

plt.subplots(figsize=(15,10))
plt.subplot(2,1,1)
plt.title('Total transactionRevenue mean by sort of next revisit time',color='b',fontsize=12)
plt.scatter(b['next_revisit_time'],b['totals_transactionRevenue'],color='orange')
#plt.xticks(b.index,rotation='horizontal',fontsize=12)
plt.yticks(fontsize=12)

plt.subplot(2,1,2)
plt.title('Total transactionRevenue mean by sort of previous revisit time',color='b',fontsize=12)
plt.scatter(b2['prev_revisit_time'],b2['totals_transactionRevenue'],color='orange')
#plt.xticks(b.index,rotation='horizontal',fontsize=12)
plt.yticks(fontsize=12)

## Analysis Visit count and Revenue relation
* Result show the ditribution have no any relation with Revenue, No need to add this feature

In [ ]:
c = train[['fullVisitorId','totals_transactionRevenue']]
c['counts'] = c['fullVisitorId']
c = c.groupby(['fullVisitorId']).agg({'totals_transactionRevenue':'mean','counts':'count'}).reset_index()
c = c.sort_values(by='counts').fillna(0).groupby('counts').mean().reset_index()
lines = c.plot.scatter(x='counts', y='totals_transactionRevenue',figsize=(15,5))

## Find the categorical feature and process data 
* We can see there are missing value  "NaN" on the data
* Find the categorical feature and visulize the data to analysis
* Analysis Unique count < 15, and unique count>15

### Find the categorical feature

In [ ]:
categorical_features_train = train.select_dtypes(include=[np.object])
categorical_features_test = test.select_dtypes(include=[np.object])
categorical_features_train.columns

### Analysis missing values for all categorical features 

* Training Set

In [ ]:
Nan_value_train = categorical_features_train.isnull().sum().sort_values(ascending=False)
Percent_Nan_value_train = Nan_value_train/categorical_features_train.shape[0]
Nan_train_data = pd.concat([Nan_value_train,Percent_Nan_value_train], axis= 1,keys = ['Missing value count','Percent'])
Nan_train_data.head(10)

* Test Set

In [ ]:
Nan_value_test = categorical_features_test.isnull().sum().sort_values(ascending=False)
Percent_Nan_value_test = Nan_value_test/categorical_features_test.shape[0]
Nan_test_data = pd.concat([Nan_value_test,Percent_Nan_value_test], axis= 1,keys = ['Missing value count','Percent'])
Nan_test_data.head(10)

## Analysis Total feature
* Analysis total feature by bropuby full ID

In [ ]:
Totals_features = [i for i in train.columns if i.find("totals")>=0 ]
Totals_features

In [ ]:
agg_dict = {}
for col in Totals_features:
    train[col] = train[col].astype('float')
    if col == 'totals_transactionRevenue':
        agg_dict[col] = 'mean'
    agg_dict[col] = "sum"
tmp = train.groupby("fullVisitorId").agg(agg_dict).reset_index()

### Plot scatter to see the distribution
* Looks the totals feature don't need to do any process, fillna(0)

In [ ]:
plt.subplots(figsize=(12,6))
plt.subplot(1,2,1)
plt.title('Revenue mean by totals hits',color='b',fontsize=12)
plt.scatter(tmp['totals_hits'],tmp['totals_transactionRevenue'],color='b')
#plt.xticks(b.index,rotation='horizontal',fontsize=12)
plt.yticks(fontsize=12)

plt.subplot(1,2,2)
plt.title('Revenue mean by totals pageviews',color='b',fontsize=12)
plt.scatter(tmp['totals_pageviews'],tmp['totals_transactionRevenue'],color='b')
#plt.xticks(b.index,rotation='horizontal',fontsize=12)
plt.yticks(fontsize=12)

In [ ]:
train['totals_pageviews']=train['totals_pageviews'].astype('float64')
train['totals_hits']=train['totals_hits'].astype('float64')
test['totals_pageviews']=test['totals_pageviews'].astype('float64')
test['totals_hits']=test['totals_hits'].astype('float64')

## Analysis unique count <15 on categorical features
* Cause we have different unique count of test set and train set, lets concate it to do one hot encoding

In [ ]:
categorical_unique = train[list(categorical_features_test)].nunique().reset_index()
categorical_unique.columns = ['features','unique_count']
categorical_unique_less_15 = categorical_unique.loc[categorical_unique['unique_count'] < 15,"features"]
categorical_unique_less_15 = list(categorical_unique_less_15)
categorical_unique_less_15

## Analysis the feature that have unique count >= 15 
* Anaysis the revenue of these feature
* Process different type feature to be training feature

In [ ]:
excluded_features = [
    'date', 'fullVisitorId', 'sessionId', 'totals.transactionRevenue', 
    'visitId', 'visitStartTime']
categorical_larger_15_feature = [i for i in categorical_features_train if i not in categorical_unique_less_15 and i not in excluded_features]
print(categorical_larger_15_feature)

### Analysis Revene correlation on geoNetwork features
* Plot the Count / Revenue correlation for all geoNetwork features

In [ ]:
geoNetwork_feature = [i for i in categorical_larger_15_feature if i.find('geoNetwork')>=0]
print(geoNetwork_feature)

In [ ]:
plt.subplots(figsize=(20,60))
for k,i in enumerate(geoNetwork_feature): 
    train_geoNetwork = train[[i]+["totals_transactionRevenue"]]
    train_geoNetwork['totals_transactionRevenue']=train_geoNetwork['totals_transactionRevenue'].fillna(0).astype('float64')
    train_geoNetwork[i+'_counts'] = train_geoNetwork[i].fillna('others')
    train_geoNetwork = train_geoNetwork.groupby(i).agg({'totals_transactionRevenue':'mean',i+'_counts':'count'}).reset_index()
    train_geoNetwork = train_geoNetwork.sort_values(by='totals_transactionRevenue',ascending=False)
    

    plt.subplot(len(geoNetwork_feature),2,(2*k)+1)
    plt.title('mean revenue by '+i,color='b',fontsize=12)
    plt.xlabel(str(i),color='b',fontsize=12)
    plt.barh(range(10),train_geoNetwork.totals_transactionRevenue[:10],color='grey')
    plt.yticks(range(10),train_geoNetwork[i][:10],rotation='horizontal',fontsize=10)
    plt.xticks(fontsize=12)
    
    plt.subplot(len(geoNetwork_feature),2,(2*k)+2)
    plt.title('Count by '+i,color='b',fontsize=12)
    plt.xlabel(str(i),color='b',fontsize=12)
    plt.barh(range(10),train_geoNetwork[i+'_counts'][:10],color='orange')
    plt.yticks(range(10),train_geoNetwork[i][:10],rotation='horizontal',fontsize=10)
    plt.xticks(fontsize=12)
    
plt.show()

### Analysis Revene correlation on device and traffic features
* Plot the Count / Revenue correlation for device and traffic features

* Device features

In [ ]:
device_feature = [i for i in categorical_larger_15_feature if i.find('device')>=0]
print(device_feature)

In [ ]:
plt.subplots(figsize=(20,40))
for k,i in enumerate(device_feature): 
    train_device = train[[i]+["totals_transactionRevenue"]]
    train_device['totals_transactionRevenue']=train_device['totals_transactionRevenue'].fillna(0).astype('float64')
    train_device[i+'_counts'] = train_device[i].fillna('others')
    train_device = train_device.groupby(i).agg({'totals_transactionRevenue':'mean',i+'_counts':'count'}).reset_index()
    train_device = train_device.sort_values(by='totals_transactionRevenue',ascending=False)
    

    plt.subplot(len(geoNetwork_feature),2,(2*k)+1)
    plt.title('mean revenue by '+i,color='b',fontsize=12)
    plt.xlabel(str(i),color='b',fontsize=12)
    plt.barh(range(10),train_device.totals_transactionRevenue[:10],color='grey')
    plt.yticks(range(10),train_device[i][:10],rotation='horizontal',fontsize=10)
    plt.xticks(fontsize=12)
    
    plt.subplot(len(geoNetwork_feature),2,(2*k)+2)
    plt.title('Count by '+i,color='b',fontsize=12)
    plt.xlabel(str(i),color='b',fontsize=12)
    plt.barh(range(10),train_device[i+'_counts'][:10],color='orange')
    plt.yticks(range(10),train_device[i][:10],rotation='horizontal',fontsize=10)
    plt.xticks(fontsize=12)
    
plt.show()

* Traffic features
Remove path and gclId feature

In [ ]:
Traffic_feature = [i for i in categorical_larger_15_feature if i.find('traffic')>=0 and i !='trafficSource_adwordsClickInfo.gclId' and i!='trafficSource_referralPath']
print(Traffic_feature)

In [ ]:
plt.subplots(figsize=(20,60))
for k,i in enumerate(Traffic_feature): 
    if i !='trafficSource_adwordsClickInfo.gclId' and i!='trafficSource_referralPath':
        train_traffic = train[[i]+["totals_transactionRevenue"]]
        train_traffic['totals_transactionRevenue']=train_traffic['totals_transactionRevenue'].fillna(0).astype('float64')
        train_traffic[i+'_counts'] = train_traffic[i].fillna('others')
        train_traffic = train_traffic.groupby(i).agg({'totals_transactionRevenue':'mean',i+'_counts':'count'}).reset_index()
        train_traffic = train_traffic.sort_values(by='totals_transactionRevenue',ascending=False)


        plt.subplot(len(geoNetwork_feature),2,(2*k)+1)
        plt.title('mean revenue by '+i,color='b',fontsize=12)
        plt.xlabel(str(i),color='b',fontsize=12)
        plt.barh(range(10),train_traffic.totals_transactionRevenue[:10],color='grey')
        plt.yticks(range(10),train_traffic[i][:10],rotation='horizontal',fontsize=10)
        plt.xticks(fontsize=12)

        plt.subplot(len(geoNetwork_feature),2,(2*k)+2)
        plt.title('Count by '+i,color='b',fontsize=12)
        plt.xlabel(str(i),color='b',fontsize=12)
        plt.barh(range(10),train_traffic[i+'_counts'][:10],color='orange')
        plt.yticks(range(10),train_traffic[i][:10],rotation='horizontal',fontsize=10)
        plt.xticks(fontsize=12)
    
plt.show()

## Analysis Revenue 
* First we analysis the assist value  of Transations Revenue
* Plot Distribution of total Revenue
* Add visiter level Revenue

In [ ]:
Revenue_assist_count = train["totals_transactionRevenue"].count()
Revenue_nonassist_count = train.shape[0]-Revenue_assist_count
label_name = ["Revenue_assist_count","Revenue_nonassist_count"]

Revenue_assist_list ={
    "Revenue":label_name,
    "Revenue_count":[Revenue_assist_count,Revenue_nonassist_count]
}

Revenue_assist_dataframe = pd.DataFrame(Revenue_assist_list)

#Revenue_non_zero = train[train["totals_transactionRevenue"]>0]["totals_transactionRevenue"]
Revenue_non_zero = train["totals_transactionRevenue"].fillna(0).reset_index().astype('float64')
Revenue_non_zero = Revenue_non_zero[Revenue_non_zero["totals_transactionRevenue"]>0]["totals_transactionRevenue"]


plt.figure(figsize=(18,18))
ax1=plt.subplot(221)
Revenue_assist_dataframe.plot(kind='pie', y = 'Revenue_count', ax=ax1,autopct='%1.1f%%',
 startangle=90, shadow=False, labels=Revenue_assist_dataframe['Revenue'], legend = False, fontsize=14)
plt.subplot(222)
sns.distplot(Revenue_non_zero)
plt.title("Distribution of Non-Zero Total Revenue")
plt.xlabel('Total TransactionRevenue')
plt.subplot(212)
sns.distplot(np.log1p(Revenue_non_zero))
plt.title("Natural Log Distribution of Non Zero Total Transactions");
plt.xlabel("Natural Log - Total Transactions");


# Future Plan
* findout the key feature on training (you can refer the upper record of different feature)
* doing esemble learning

In [ ]:
train.head()

In [ ]:
test.head()